## Home Work 2 
### Part 2 (Implementation Questions)
Name: Khalid Saifullah \
ID: A20423546 \
Semester: Spring 2021 \
Date: 17th March 2021

In [1]:
#Importing the libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from scipy.special import expit

In [2]:
######Comment or uncomment this cell to run the iris dataset!!!######


#Loading the iris dataset
iris = pd.read_csv("Iris.csv")
iris = iris.sample(frac=1).reset_index(drop=True) # Shuffle

# Separating independent and dependent variables
X = np.array(iris[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]) #Ignored the ID column
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) #min-max normalisation used for the features
Y = iris.Species #Only selected the last column i.e. the species column
one_hot_encoder = OneHotEncoder(sparse=False) #Shuffled to remove any biases due to ordering
Y = one_hot_encoder.fit_transform(np.array(Y).reshape(-1, 1)) #Returns one hot encoded array of the 3 classes

In [3]:
######Comment or uncomment this cell to run the wine dataset!!!######


#Loading the wine dataset
#wine = pd.read_csv("wine.csv")
#wine = wine.sample(frac=1).reset_index(drop=True) # Shuffle

#Searching for any missing values in the dataset
#wine.isnull().sum() #No missing values found - preproscessing step

# Separating independent and dependent variables
#X = wine.drop('Types',axis=1)
#X = np.array(wine[['Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315 of diluted wines','Proline']])
#X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) #min-max normalisation used for the features
#Y = wine['Types']
#one_hot_encoder = OneHotEncoder(sparse=False)
#Y = one_hot_encoder.fit_transform(np.array(Y).reshape(-1, 1)) #Returns one hot encoded array of the 3 classes

In [4]:
#Splitted our dataset for training, testing and validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2)

In [5]:
#"nodes" is a list of integers. Each integer denotes the number of nodes in each layer. 
#The length of this list denotes the number of layers.

def NeuralNetwork(X_train, Y_train, X_val=None, Y_val=None, epochs=10, nodes=[], lr=0.15): #epochs by default set to 10
    hidden_layers = len(nodes) - 1 #Subtracting 1 becaues of the output layer which is included in the len(nodes) value
    weights = InitializeWeights(nodes) #initiasing the weights in all the nodes

    for epoch in range(1, epochs+1):
        weights = Train(X_train, Y_train, lr, weights)
        #printing testing accuracies every 10 epochs
        if(epoch % 10 == 0):
            print("Epoch: {}".format(epoch))
            print("Training Accuracy: {}".format(Accuracy(X_train, Y_train, weights)))
            if X_val.any():
                print("Validation Accuracy: {}".format(Accuracy(X_val, Y_val, weights)))
            
    return weights

In [6]:
#Each element in the weights list represents a hidden layer and holds the weights 
#of connections from the previous layer (including the bias) to the current layer.
#So, element i in weights holds the weights of the connections from layer i-1 to layer i.
#Note that the input layer has no incoming connections so it is not present in weights.

#Initialising weights with random values in the range -1 to 1.
def InitializeWeights(nodes):
    layers, weights = len(nodes), []
    
    for i in range(1, layers):
        w = [[np.random.uniform(-1, 1) for k in range(nodes[i-1] + 1)] #Try using: w = [[np.random.normal(-1, 1) for k in range(nodes[i-1] + 1)]
              for j in range(nodes[i])]
        weights.append(np.matrix(w))
    
    return weights

In [7]:
#Forward propagation
def ForwardPropagation(x, weights, layers):
    activations, layer_input = [x], x
    for j in range(layers):
        activation_out = Sigmoid(np.dot(layer_input, weights[j].T))
        activations.append(activation_out)
        layer_input = np.append(1, activation_out) # Augmenting with the bias term
    
    return activations

In [8]:
#Backward propagation
def BackPropagation(y, activations, weights, layers):
    outputFinal = activations[-1]
    error = np.matrix(y - outputFinal) # Error at output
    
    for j in range(layers, 0, -1): #increments by -1 i.e. we are traversing from the last layer
        currentActivation = activations[j]
        
        if(j > 1):
            # Augmenting previous activation
            previousActivation = np.append(1, activations[j-1])
        else:
            # First hidden layer, prevActivation is input (without bias)
            previousActivation = activations[0]
        
        del_value = np.multiply(error, SigmoidDerivative(currentActivation)) #calculating the delta value
        weights[j-1] += lr * np.multiply(del_value.T, previousActivation) #updating the weights

        w = np.delete(weights[j-1], [0], axis=1) # Removing bias from weights
        error = np.dot(del_value, w) # Calculate error for current layer
    
    return weights

In [9]:
#Training our model
def Train(X, Y, lr, weights):
    layers = len(weights)
    for i in range(len(X)):
        x, y = X[i], Y[i]
        x = np.matrix(np.append(1, x)) # Augmenting the feature vector
        
        activations = ForwardPropagation(x, weights, layers)
        weights = BackPropagation(y, activations, weights, layers)

    return weights

In [10]:
#Defining the sigmoid activation function and its derivative
def Sigmoid(x):
    return expit(x)

def SigmoidDerivative(x):
    return np.multiply(x, 1-x)

In [11]:
#Predicting here
def Predict(sample, weights):
    layers = len(weights)
    sample = np.append(1, sample) # Augmenting the feature vector
    
    #Forward Propagation
    activations = ForwardPropagation(sample, weights, layers)
    
    outputFinal = activations[-1].A1
    index = FindMaxActivation(outputFinal)

    #Initializing our prediction vector to zero
    y = [0 for i in range(len(outputFinal))]
    y[index] = 1  # Set guessed class to 1

    return y # Return the prediction vector


#Finding the most probable class
def FindMaxActivation(output):
    m, index = output[0], 0
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
    
    return index

In [12]:
#Computing the model accuracy
def Accuracy(X, Y, weights):
    """Run set through network, find overall accuracy"""
    score = 0 #initialising our scorecard
    for i in range(len(X)):
        OurPrediction = Predict(X[i], weights)

        if(list(Y[i]) == OurPrediction):
            score += 1 # correctly predicted

    return score / len(X)

In [13]:
f = len(X[0]) # Number of features
o = len(Y[0]) # Number of outputs or classes

#Change the hidden nodes, learning rates and epochs below to fine tune the model hyperparameters
#Use hiddenlayer1=4 and hiddenlayer=8 for iris dataset
#Use hiddenlayer1=8 and hiddenlayer=8 for iris dataset
hiddenlayer1 = 4
hiddenlayer2 = 8
layers = [f, hiddenlayer1, hiddenlayer2, o] # Number of nodes in layers. Note the two hidden layers in the middle
lr, epochs = 0.1, 90

weights = NeuralNetwork(X_train, Y_train, X_val, Y_val, epochs=epochs, nodes=layers, lr=lr);

Epoch: 10
Training Accuracy: 0.3888888888888889
Validation Accuracy: 0.2222222222222222
Epoch: 20
Training Accuracy: 0.37962962962962965
Validation Accuracy: 0.2222222222222222
Epoch: 30
Training Accuracy: 0.3611111111111111
Validation Accuracy: 0.2222222222222222
Epoch: 40
Training Accuracy: 0.6574074074074074
Validation Accuracy: 0.7777777777777778
Epoch: 50
Training Accuracy: 0.7407407407407407
Validation Accuracy: 0.7777777777777778
Epoch: 60
Training Accuracy: 0.8518518518518519
Validation Accuracy: 0.8888888888888888
Epoch: 70
Training Accuracy: 0.9351851851851852
Validation Accuracy: 0.9629629629629629
Epoch: 80
Training Accuracy: 0.9629629629629629
Validation Accuracy: 0.9629629629629629
Epoch: 90
Training Accuracy: 0.9814814814814815
Validation Accuracy: 0.9629629629629629


In [14]:
print("Testing Accuracy: {}".format(Accuracy(X_test, Y_test, weights)))

Testing Accuracy: 0.7333333333333333
